# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [118]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [119]:
cities_df = pd.read_csv('../output_data/cities.csv')

cities_df.head()


,Unnamed: 0,City Name,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hofn,64.25,-15.21,54.32,85,100,13.65,IS,1593296014
1,1,Rikitea,-23.12,-134.97,70.54,75,0,17.16,PF,1593295847
2,2,Busselton,-33.65,115.33,60.01,75,100,22.15,AU,1593295847
3,3,Huilong,31.81,121.66,78.01,99,100,1.99,CN,1593296078
4,4,Idrinskoye,54.37,92.14,58.60,86,3,2.53,RU,1593296078


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [120]:
# Configure gmaps
gmaps.configure(api_key=g_key)

 # set coordinates for cities and use Humidity as the weight
city_coord = cities_df[['Lat', 'Lng']]
humidity_weight = cities_df['Humidity'].astype(float)

# create a figure
figure_layout = {
    'width': '1200px',
    'height': '550px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=(0,0), zoom_level=2.2, layout=figure_layout)

# add the heat layer
heat_layer = gmaps.heatmap_layer(city_coord, weights=humidity_weight, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

# render
fig


Figure(layout=FigureLayout(border='1px solid black', height='550px', padding='1px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [121]:
ideal_cities = cities_df.loc[(cities_df['Humidity'] < 70) & (cities_df['Temp'] < 80) & (cities_df['Temp'] > 70) & (cities_df['Cloudiness'] < 20)]
ideal_cities

,Unnamed: 0,City Name,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,Fort-Shevchenko,44.51,50.26,70.45,53,0,5.48,KZ,1593296080
257,257,Athens,37.98,23.72,78.01,44,0,7.14,GR,1593295740
275,275,Kumul,42.80,93.45,73.20,39,0,3.36,CN,1593296112
324,324,Awjilah,29.11,21.29,77.86,45,0,11.12,LY,1593296114
385,385,Loukísia,38.48,23.45,75.99,52,0,5.82,GR,1593296127
470,470,Mahajanga,-15.72,46.32,71.60,68,0,4.70,MG,1593296138
471,471,Nushki,29.55,66.02,76.51,17,1,4.12,PK,1593296138
520,520,Tadine,-21.55,167.88,71.58,62,16,11.01,NC,1593296144
530,530,Beli Potok,44.71,20.52,73.40,60,3,8.05,RS,1593296146
531,531,Gătaia,45.43,21.43,78.80,44,0,17.22,RO,1593296146


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [122]:
# hname = result['results'][0]['name']
# hlat = result['results'][0]['geometry']['location']['lat']
# hlng = result['results'][0]['geometry']['location']['lng']
# print(f'Hotel {hname} is located at {hlat} by {hlng}')


In [123]:

# format ideal city lat/lng for Places API
ideal_city_coords = ideal_cities['Lat'].astype(str) + ', ' + ideal_cities['Lng'].astype(str)

# to build DataFrame later
hotel_name = []
hotel_lat = []
hotel_lng = []
 
# geocoordinates
target_coordinates = ""
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# parameters
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# loop through cities and make requests using parameter dictionary
for latlng in ideal_city_coords:

    params['location'] = latlng

    try:
        response = requests.get(base_url, params=params)
        hotel_result = response.json()

        hotel_name.append(hotel_result['results'][0]['name'])
        hotel_lat.append(hotel_result['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(hotel_result['results'][0]['geometry']['location']['lng'])

    except:
        hotel_name.append('No Hotel Located')
        hotel_lat.append('0')
        hotel_lng.append('0')

In [124]:
# create new df for gmap markers
ideal_city_names = ideal_cities['City Name']
ideal_countries = ideal_cities['Country']

ideal_hotels = pd.DataFrame({
    'Hotel Name': hotel_name,
    'City': ideal_city_names,
    'Country': ideal_countries,
    'Lat': hotel_lat,
    'Lng': hotel_lng
})

#ideal_hotels.head()

ideal_hotels.dtypes


Hotel Name    object
City          object
Country       object
Lat           object
Lng           object
dtype: object

In [125]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_hotels.iterrows()]
locations = ideal_hotels[["Lat", "Lng"]].astype('float')

In [130]:
#Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='550px', padding='1px', width='1200px'))